# sdata usage

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 0
    
import logging
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.WARNING, datefmt='%I:%M:%S')

import os
import sys
import numpy as np
import pandas as pd
import sdata
import uuid
print("sdata v{}".format(sdata.__version__))

## create a Data object

In [ ]:
data = sdata.Data(name="data", uuid="38b26864e7794f5182d38459bab85841")
data.metadata.add("my_key", 123, unit="m^3", description="a volume", label="V")
data.metadata.add("force", 1.234, unit="kN", description="x force", label="F_x")
data

In [ ]:
data.metadata.df

In [ ]:
data.uuid

In [ ]:
#help(data)

## create a hierarchy of objects

In [ ]:
group1 = sdata.Data(name="group1", uuid="dbc894745fb04f7e87a990bdd4ba97c4")
print(group1)

In [ ]:
data1 = sdata.Data(name="data1", uuid="38b26864e7794f5182d38459bab8584f")
data2 = sdata.Data(name="data2", uuid="b1fd2643-af55-4b33-b044-22070a0dc7c7")
data3 = sdata.Data(name="data3", uuid=uuid.UUID("664577c2d3134b598bc4d6c13f20b71a"))

group1.add_data(data1)
group1.add_data(data2)
group1.add_data(data3)
print(group1.group)

In [ ]:
group1.keys()

In [ ]:
group1.values()

## working with Data

data1a = group1.get_data_by_uuid(uid="38b26864e7794f5182d38459bab8584f")
assert data1a.name == "data1"
assert data1a.uuid == "38b26864e7794f5182d38459bab8584f"

data3a = group1.get_data_by_uuid(uid="664577c2d3134b598bc4d6c13f20b71a")
assert data3a.name == "data3"
assert data3a.uuid == "664577c2d3134b598bc4d6c13f20b71a"

In [ ]:
data1b = group1.get_data_by_name("data1")
assert data1b.name == "data1"
assert data1b.uuid == "38b26864e7794f5182d38459bab8584f"
data1b

## Data IO

### Folder IO

In [ ]:
group1.to_folder(path="/tmp/b/", dtype="xlsx")

In [ ]:
group1.tree_folder("/tmp/b")

In [ ]:
group1.to_folder(path="/tmp/a/", dtype="csv")

In [ ]:
group1.tree_folder("/tmp/a")

In [ ]:
group1_folder = sdata.Data.from_folder(path="/tmp/a")
print("!", group1_folder)
assert (group1.name==group1_folder.name)
assert (group1.uuid==group1_folder.uuid)
print(group1_folder.dir())
group1_folder

### Excel IO (wip)

In [ ]:
data.to_xlsx(filepath="/tmp/data1.xlsx")
data_xlsx = sdata.Data.from_xlsx(filepath="/tmp/data1.xlsx")
print(data.name, data_xlsx.name)
print(data.uuid, data_xlsx.uuid)
#assert data.name==data_xlsx.name
data_xlsx

## inspect Data

In [ ]:
group1.dir()

In [ ]:
[x for x in dir(data) if not x.startswith("_")]

### Data with comments

In [ ]:
datac = sdata.Data(name="data", uuid="38b26864e7794f5182d38459bab85842", description="this is remarkable")
datac

In [ ]:
datac.description += """\n\nhere is your comment\nwhat's next?"""

In [ ]:
print(datac.description)

In [ ]:
datac.to_xlsx("/tmp/data_with_description.xlsx")

In [ ]:
datac2 = sdata.Data.from_xlsx("/tmp/data_with_description.xlsx")
print(datac2.description)

## make a copy

In [ ]:
data = sdata.Data(name="data", uuid="38b26864e7794f5182d38459bab85842", description="this is remarkable")
datac = data.copy()
print("data  {0.uuid}".format(data))
print("datac {0.uuid}".format(datac))
print("datac.metadata['!sdata_parent'] {0.value}".format(datac.metadata["!sdata_parent"]))


In [ ]:
datac.metadata.df

## data from url

In [ ]:
url = "https://gist.github.com/lepy/fdbdce805b206322d8013a1375da0eb9/raw/76e6dad16e78befbbd711e074e7fb8493d15be89/medmnsteel_0.01mps_1_rd.json"
data = sdata.Data.from_url(url, stype="json")
data.describe()
data.metadata.df